In [ ]:
from music21 import converter, instrument, clef, stream, analysis, midi, stream
from plotters import saveToMidi

In [ ]:
# TODO: Save phrases within a stream (instead of pickle)

bassoon = instrument.Bassoon()
print(bassoon.lowestNote.ps, bassoon.highestNote)

toy = converter.parse("tinyNotation: 4/4 r8 d' d' d' g' a' b' a' g2 r")

def shiftOctave(s: stream.Stream, o: int):
    for n in s.recurse().notes:
        n.octave += o

p = analysis.discrete.Ambitus()
pitchMin, pitchMax = p.getPitchSpan(toy)
print(pitchMin)

shiftOctave(toy, -2)
toy.show()

sp = midi.realtime.StreamPlayer(toy)
sp.play()

print(pitchMin)

In [ ]:
score = converter.parse("../eroica.mxl")
#score.measures(1,6).show()
#saveToMidi(score, "../eroica.mid")
score.write(fmt="musicxml", fp=f"../eroica.musicxml")

# TODO: Check what MIDI sounds like after arrangement (transposing instruments etc.)

WindowsPath('c:/Users/Lucas/OneDrive - Durham University/PHYS4213/eroica.musicxml')

In [22]:
score.write(fmt="musicxml", fp=f"../raiders")

musicxml.m21ToXml: WARNING: we are out of midi channels! help!


WindowsPath('c:/Users/Lucas/OneDrive - Durham University/PHYS4213/raiders.musicxml')

In [ ]:
score = converter.parse("../Pickles/Beethoven/Beethoven_score.musicxml")
#saveToMidi(score, "../Pickles/Beethoven/beethoven.mid")

new = stream.Score()

for part in score.parts:

    print(part.partName)
    part.atSoundingPitch = True
    print(part.atSoundingPitch)

    voices = part.voicesToParts()
    print(len(voices.parts))
    if len(voices.parts) > 1:
        part = part.voicesToParts().parts[0]

    new.append(part)

new.measures(1,7).show()
saveToMidi(new, "../Pickles/Beethoven/sounding_pitch.mid")
#new.write(fmt="musicxml", fp=f"../Pickles/Beethoven/beethoven.musicxml")

In [ ]:
saveToMidi(new, "../Pickles/Beethoven/beethoven.mid")

In [ ]:
print(len(score.parts[2].voices))
cl = score.parts[2].voicesToParts()
vl = score.parts[8].voicesToParts()
print(len(cl.parts))
print(len(vl.parts))
#score.show("text")
#cl.show("text")

#saveToMidi(cl, "../Pickles/Beethoven/cl.mid")

In [ ]:
score.flatten().notes.stream().show("midi")